In [17]:
import numpy as np
import random

data = [
    [20, 16, 55.7], [25, 12, 59.3], [16, 30, 5.6], [15, 14, 30.1],
    [27, 10, 43.4], [12, 19, 18.61], [19, 12, 22.1], [28, 8, 47.3],
    [35, 6, 76.4], [20, 17, 20.9]
]
train_data = np.array(data[:8])
test_data = np.array(data[8:])

def membership(x, a, b, c, shape="sigmoid"):
    if shape == "sigmoid":
        a = np.clip(a, -10, 10)  # Membatasi nilai 'a'
        return 1 / (1 + np.exp(-a * (x - b))) * c
    elif shape == "triangular":
        return max(0, min((x - a) / (b - a), (c - x) / (c - b)))

def fuzzy_inference(temp, humidity, params):
    temp_params = params[:6]
    humidity_params = params[6:12]
    duration_params = params[12:]

    temp_low = membership(temp, temp_params[0], temp_params[1], 1)
    temp_med = membership(temp, temp_params[2], temp_params[3], 1)
    temp_high = membership(temp, temp_params[4], temp_params[5], 1)

    humidity_low = membership(humidity, humidity_params[0], humidity_params[1], 1)
    humidity_med = membership(humidity, humidity_params[2], humidity_params[3], 1)
    humidity_high = membership(humidity, humidity_params[4], humidity_params[5], 1)

    short_duration = duration_params[0]
    long_duration = duration_params[1]

    rule1 = temp_high * humidity_low * long_duration
    rule2 = temp_med * humidity_med * short_duration
    rule3 = temp_low * humidity_high * short_duration

    total_duration = rule1 + rule2 + rule3
    return total_duration

def calculate_fitness(individual):
    total_error = 0
    for data_point in train_data:
        temp, humidity, actual_duration = data_point
        predicted_duration = fuzzy_inference(temp, humidity, individual)
        total_error += abs(predicted_duration - actual_duration)

    return total_error / len(train_data)

def genetic_algorithm(population_size, num_generations, mutation_rate):
    population = [np.random.uniform(low=0, high=50, size=14) for _ in range(population_size)]

    for generation in range(num_generations):
        fitness_scores = [calculate_fitness(individual) for individual in population]

        sorted_population = [ind for _, ind in sorted(zip(fitness_scores, population), key=lambda x: x[0])]
        best_individuals = sorted_population[:population_size // 2]

        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = random.sample(best_individuals, 2)
            crossover_point = random.randint(1, 13)
            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            new_population.append(child)

        for individual in new_population:
            if random.random() < mutation_rate:
                mutation_idx = random.randint(0, 13)
                individual[mutation_idx] = np.random.uniform(0, 50)

        population = best_individuals + new_population

    best_individual = sorted_population[0]
    best_fitness = calculate_fitness(best_individual)
    return best_individual, best_fitness

In [18]:
if __name__ == "__main__":
    population_size = 50
    num_generations = 40
    mutation_rate = 0.2

    best_params, best_error = genetic_algorithm(population_size, num_generations, mutation_rate)

    print(f"Best Parameters: {best_params}")
    print(f"Best Fitness: {best_error}")

    for test in test_data:
        temp, humidity, actual_duration = test
        predicted_duration = fuzzy_inference(temp, humidity, best_params)
        print(f"Actual: {actual_duration}, Predicted: {predicted_duration}")

Best Parameters: [14.99185381 21.13975331 49.18993101  5.69269691  4.52424634 27.41316115
 10.84606761  0.34392741  2.70316637 10.89406797 19.93145389  4.61251291
 29.99435514 17.70205163]
Best Fitness: 9.767460186969634
Actual: 76.4, Predicted: 47.696432399738356
Actual: 20.9, Predicted: 29.994689729740998
